In [131]:
import json
import os
import re

import openrouteservice as ors
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

from geopy.geocoders import Nominatim
from scipy.interpolate import Rbf

In [132]:
# Set your city manually here
city = "Fort Worth, TX"  # Change this as needed

# Set max walking time
max_walking_time = 60  # in minutes

# Set max price
max_price = 1250  # in dollars

In [133]:
# Define the pattern for the city (case insensitive)
city_pattern = re.compile(re.escape(city), re.IGNORECASE)

# Open and read the file
with open("listings.txt", "r") as file:
    data = file.read()

# Find all lines that contain the city pattern
matches = [line for line in data.splitlines() if city_pattern.search(line)]

# Organize data by address (set to store unique addresses)
unique_addresses = []
address_prices = {}  # To store prices for each address

# Go through each match and process data
for match in matches:
    # If there's a vertical bar '|', get everything after it (excluding space)
    if '|' in match:
        match = match.split('|', 1)[1].lstrip()  # Extract after the bar, remove leading spaces

    # Add to list of unique addresses
    if match not in unique_addresses:
        unique_addresses.append(match)
        address_prices[match] = set()  # Use a set to prevent duplicate prices

# Iterate through the file again, this time collecting prices for each address
current_address = None

# Extract prices from the data
for line in data.splitlines():
    # Check if the line contains any of the unique addresses
    for address in unique_addresses:
        if address in line:
            current_address = address  # Set current address when we find it
            break

    if current_address:  # If we've set a current address, check for prices
        price_matches = re.findall(r'\$(\d{1,3}(?:,\d{3})*)', line)  # Match prices
        
        if price_matches:
            for price in price_matches:
                address_prices[current_address].add(price)  # Use set to avoid duplicates

# Now, convert the address_prices dictionary into a DataFrame
rows = []
columns = ['Address', 'Price']

# Prepare rows for the DataFrame
for address, prices in address_prices.items():
    for price in prices:
        rows.append([address, price])

df = pd.DataFrame(rows, columns=columns)

In [134]:
pd.set_option('display.max_colwidth', None)

df.head()

,Address,Price
0,"6351 Hulen Bend Blvd, Fort Worth, TX","1,247"
1,"6351 Hulen Bend Blvd, Fort Worth, TX",795
2,"6751 Westcreek Dr, Fort Worth, TX",799
3,"3500 Renzel Blvd, Fort Worth, TX",845
4,"3500 Renzel Blvd, Fort Worth, TX","1,199"


In [135]:
# OpenRouteService Client
client = ors.Client(key='5b3ce3597851110001cf624817e35411a1444fc7901b0872949c4b86')

# Initialize Geolocator for address geocoding
geolocator = Nominatim(user_agent="address_optimizer")

# Fixed intermediate addresses (coordinates)
intermediate_coords = [
    (32.74840963982297, -97.32497439905646),  # Address 1
    (32.747705801640954, -97.35808003618311),  # Address 2
    (32.7515678225884, -97.35133168289146)     # Address 3
]

# Function to calculate the centroid of the intermediate locations
def calculate_centroid(coords):
    latitudes = [coord[0] for coord in coords]
    longitudes = [coord[1] for coord in coords]
    return sum(latitudes) / len(coords), sum(longitudes) / len(coords)

# Calculate the centroid
centroid_lat, centroid_lon = calculate_centroid(intermediate_coords)

# Define cache file and load cache if it exists.
cache_file = 'walking_times_cache.json'
if os.path.exists(cache_file):
    with open(cache_file, 'r') as f:
        cache = json.load(f)
else:
    cache = {}

# List to store results for each address
walk_times = []

# Iterate over each address in the DataFrame 'df'
for index, row in df.iterrows():
    address = row['Address']
    # Create a unique key for the address and centroid
    key = f"{address}_{centroid_lat}_{centroid_lon}"
    
    if key in cache:
        walking_time_minutes = cache[key]
    else:
        # Geocode the address
        location = geolocator.geocode(address)
        if location:
            address_lat = location.latitude
            address_lon = location.longitude
            
            # Request walking route from the address to the centroid
            route = client.directions(
                coordinates=[(address_lon, address_lat), (centroid_lon, centroid_lat)], 
                profile='foot-walking', 
                format='geojson'
            )
            # Extract walking time (in seconds) then convert to minutes
            if route.get('features'):
                walking_time_seconds = route['features'][0]['properties']['segments'][0]['duration']
                walking_time_minutes = walking_time_seconds / 60
            else:
                walking_time_minutes = None
        else:
            walking_time_minutes = None
        
        # Save result in cache
        cache[key] = walking_time_minutes

        # Save updated cache to file
        with open(cache_file, 'w') as f:
            json.dump(cache, f, indent=2)
    
    # Append result to list
    walk_times.append({
        'Address': address,
        'Price': row['Price'],
        'Walking Time to Centroid (minutes)': walking_time_minutes
    })

# Convert results into a DataFrame for better readability
walk_times_df = pd.DataFrame(walk_times)

In [136]:
# Remove commas from the 'Price' column and convert it to numeric
walk_times_df['Price'] = walk_times_df['Price'].str.replace(',', '').astype(float)

walk_times_df = walk_times_df[walk_times_df['Walking Time to Centroid (minutes)'] < max_walking_time]
walk_times_df = walk_times_df[walk_times_df['Price'] < max_price]

# Sort the DataFrame based on 'Walking Time to Centroid (minutes)'
walk_times_df_sorted = walk_times_df.sort_values(by='Walking Time to Centroid (minutes)', ascending=True)

# Save the sorted DataFrame to a new CSV file
walk_times_df_sorted.to_csv('walk_times_sorted.csv', index=False)

# Display the DataFrame
walk_times_df_sorted

,Address,Price,Walking Time to Centroid (minutes)
43,"1001 W 7th St, Fort Worth, TX",1190.0,16.696667
41,"1012 Burnett St, Fort Worth, TX",1184.0,19.608333
97,"Magnolia on Park, 2901 W 5th St APT 302, Fort Worth, TX 76107",1149.0,24.000000
36,"929 Norwood St, Fort Worth, TX",1150.0,27.528333
39,"3125 Sondra Dr, Fort Worth, TX",1175.0,34.093333
40,"3125 Sondra Dr, Fort Worth, TX",1182.0,34.093333
31,"555 Elm St, Fort Worth, TX",1106.0,34.726667
94,"520 Samuels Ave, Fort Worth, TX",1143.0,37.763333
50,"320 E Broadway Ave, Fort Worth, TX",1234.0,38.905000
79,"3529 W 7th St, Fort Worth, TX",900.0,39.606667


In [137]:
walk_times_df_sorted = pd.read_csv('walk_times_sorted.csv')

In [138]:
# Compute Dominate Metric
walk_times_df_sorted['Dominated Count'] = 0
walk_times_df_sorted['Dominates Count'] = 0
n = len(walk_times_df_sorted)
for i in range(n):
    for j in range(n):
        if i != j:
            if (walk_times_df_sorted.iloc[i]['Price'] < walk_times_df_sorted.iloc[j]['Price'] and
                walk_times_df_sorted.iloc[i]['Walking Time to Centroid (minutes)'] < walk_times_df_sorted.iloc[j]['Walking Time to Centroid (minutes)']):
                walk_times_df_sorted.at[i, 'Dominates Count'] += 1
                walk_times_df_sorted.at[j, 'Dominated Count'] += 1

# Avoid division by zero, set default ratio to a large value when not dominated
walk_times_df_sorted['Dominate/Dominated Ratio'] = walk_times_df_sorted['Dominates Count'] / walk_times_df_sorted['Dominated Count'].replace(0, 1)

# Normalize Dominate Count and Dominate/Dominated Ratio
walk_times_df_sorted['Normalized Dominates Count'] = (walk_times_df_sorted['Dominates Count'] - walk_times_df_sorted['Dominates Count'].min()) / (walk_times_df_sorted['Dominates Count'].max() - walk_times_df_sorted['Dominates Count'].min())
walk_times_df_sorted['Normalized Dominate/Dominated Ratio'] = (walk_times_df_sorted['Dominate/Dominated Ratio'] - walk_times_df_sorted['Dominate/Dominated Ratio'].min()) / (walk_times_df_sorted['Dominate/Dominated Ratio'].max() - walk_times_df_sorted['Dominate/Dominated Ratio'].min())

# Compute statistical outlier penalty
z_price = (walk_times_df_sorted['Price'] - walk_times_df_sorted['Price'].mean()) / walk_times_df_sorted['Price'].std()
z_time = (walk_times_df_sorted['Walking Time to Centroid (minutes)'] - walk_times_df_sorted['Walking Time to Centroid (minutes)'].mean()) / walk_times_df_sorted['Walking Time to Centroid (minutes)'].std()
walk_times_df_sorted['Outlier Penalty'] = (z_price**2 + z_time**2) ** 0.5
walk_times_df_sorted['Normalized Outlier Penalty'] = (walk_times_df_sorted['Outlier Penalty'] - walk_times_df_sorted['Outlier Penalty'].min()) / (walk_times_df_sorted['Outlier Penalty'].max() - walk_times_df_sorted['Outlier Penalty'].min())

# Compute Dominate Metric with Outlier Penalty
walk_times_df_sorted['Dominate Metric'] = (walk_times_df_sorted['Normalized Dominates Count'] * walk_times_df_sorted['Normalized Dominate/Dominated Ratio']) + (1 - walk_times_df_sorted['Normalized Outlier Penalty']) * 0.5

# Create the scatter plot
fig = px.scatter(
    walk_times_df_sorted, 
    x='Price', 
    y='Walking Time to Centroid (minutes)', 
    hover_name='Address',
    title='Price vs Walking Time',
    labels={'Price': 'Price ($)', 'Walking Time to Centroid (minutes)': 'Walking Time (minutes)'},
    color='Dominate Metric',
    color_continuous_scale='Viridis',
    template='plotly_dark'
)

# Set margins and grid range
x_margin = (walk_times_df_sorted['Price'].max() - walk_times_df_sorted['Price'].min()) * 0.05
y_margin = (walk_times_df_sorted['Walking Time to Centroid (minutes)'].max() - walk_times_df_sorted['Walking Time to Centroid (minutes)'].min()) * 0.05
x_min = 500
x_max = walk_times_df_sorted['Price'].max() + x_margin
y_min = 0
y_max = walk_times_df_sorted['Walking Time to Centroid (minutes)'].max() + y_margin

fig.update_layout(
    xaxis=dict(type='linear', title='Price ($)', range=[x_min, x_max]),
    yaxis=dict(title='Walking Time (minutes)', range=[y_min, y_max]),
    xaxis_gridcolor='rgba(255, 255, 255, 0.05)',
    yaxis_gridcolor='rgba(255, 255, 255, 0.05)',
    xaxis_zerolinecolor='rgba(255, 255, 255, 0.5)',
    yaxis_zerolinecolor='rgba(255, 255, 255, 0.5)'
)

# Create grid for interpolation over full plot area
price_range = np.linspace(x_min, x_max, 200)
time_range = np.linspace(y_min, y_max, 200)
price_grid, time_grid = np.meshgrid(price_range, time_range)

# Use Rbf for continuous interpolation
points = walk_times_df_sorted[['Price', 'Walking Time to Centroid (minutes)']].values
values = walk_times_df_sorted['Dominate Metric'].values

try:
    rbf = Rbf(points[:,0], points[:,1], values, function='inverse', smooth=0)
except np.linalg.LinAlgError:
    rbf = Rbf(points[:,0], points[:,1], values, function='inverse', smooth=1)

grid_z = rbf(price_grid, time_grid)

# Add translucent contour layer with hover disabled
fig.add_trace(go.Contour(
    x=price_range,
    y=time_range,
    z=grid_z,
    colorscale='Viridis',
    opacity=0.5,
    contours=dict(showlines=False),
    showscale=False,
    name='Dominate Metric Distribution',
    hoverinfo='skip'
))

# Update marker to add a thin border around each point and set size to 8
fig.update_traces(
    marker=dict(
        size=10,
        opacity=1, 
        line=dict(width=1, color='DarkSlateGrey')
    ),
    selector=dict(mode='markers')
)

fig.show()

In [139]:
fig.write_html("pareto.html")

In [140]:
walk_times_df_sorted = walk_times_df_sorted.sort_values(by='Dominated Count', ascending=False)

walk_times_df_sorted

,Address,Price,Walking Time to Centroid (minutes),Dominated Count,Dominates Count,Dominate/Dominated Ratio,Normalized Dominates Count,Normalized Dominate/Dominated Ratio,Outlier Penalty,Normalized Outlier Penalty,Dominate Metric
14,"The Berkeley, 2001 Park Hill Dr #13306-2, Fort Worth, TX 76110",1229.0,56.000000,13,0,0.000000,0.000000,0.000000,1.920816,0.812919,0.093541
10,"Ramble & Rose Apartments, 501 W Rosedale St #449, Fort Worth, TX 76104",1228.0,45.000000,9,1,0.111111,0.166667,0.018519,1.214106,0.483151,0.261511
8,"320 E Broadway Ave, Fort Worth, TX",1234.0,38.905000,8,0,0.000000,0.000000,0.000000,1.070172,0.415988,0.292006
4,"3125 Sondra Dr, Fort Worth, TX",1175.0,34.093333,2,3,1.500000,0.500000,0.250000,0.536843,0.167125,0.541438
5,"3125 Sondra Dr, Fort Worth, TX",1182.0,34.093333,2,3,1.500000,0.500000,0.250000,0.598806,0.196038,0.526981
3,"929 Norwood St, Fort Worth, TX",1150.0,27.528333,1,5,5.000000,0.833333,0.833333,0.840405,0.308774,1.040058
7,"520 Samuels Ave, Fort Worth, TX",1143.0,37.763333,1,3,3.000000,0.500000,0.500000,0.178685,0.000000,0.750000
11,"1115 S Jennings Ave APT 11, Fort Worth, TX 76104",1045.0,45.000000,1,1,1.000000,0.166667,0.166667,1.055293,0.409045,0.323255
12,"701 N Hampton St, Fort Worth, TX",1029.0,46.085000,1,1,1.000000,0.166667,0.166667,1.235281,0.493032,0.281262
13,"4023 Mattison Garage Apartment, Fort Worth, TX 76107",950.0,55.000000,1,1,1.000000,0.166667,0.166667,2.321742,1.000000,0.027778
